In [24]:
import pandas as pd
import sqlite3
import numpy as np

In [25]:
#Configuração do pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Bronze

In [26]:
# Leitura de arquivo CSV
# Conversão de csvpara parquet 
ac = pd.read_csv('./medallion/bronze/acidentes2025.csv',sep=';' ,index_col="id" ,encoding='latin-1')
dt = pd.read_csv('./medallion/bronze/datatran2025.csv', sep=';' ,index_col="id" ,encoding='latin-1') 

ac.to_csv('./medallion/silver/acidentes2025.csv')
dt.to_csv('./medallion/silver/datatran2025.csv')

# Silver

In [27]:
#Leitura dos arquivos Parquet
ac_silver = pd.read_csv('./medallion/silver/acidentes2025.csv')
dt_silver = pd.read_csv('./medallion/silver/datatran2025.csv')

In [28]:
#Novo data frame
df = ac_silver.merge(dt_silver, how='inner')

verificações

In [29]:
#Verificação de dados faltantes
df.isna().sum()[df.isna().sum() > 0].sort_values(ascending=False)

tipo_veiculo      518
tipo_envolvido    108
estado_fisico     108
sexo              108
uop                 4
delegacia           2
dtype: int64

In [ ]:
df.loc[df.isna().sum()[df.isna().sum() > 0]]

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop,pessoas,ignorados,feridos,veiculos
518,661704,1492141,2025-02-16,domingo,08:00:00,ES,101,"448,2",MIMOSO DO SUL,Velocidade Incompatível,Saída de leito carroçável,Com Vítimas Feridas,Pleno dia,Crescente,Sol,Simples,Reta,Não,1190503,Caminhão-trator,M.BENZ/AXOR 1933 LS,2022,Condutor,Lesões Graves,48,Masculino,0,0,1,0,"-21,13028101","-41,28044011",SPRF-ES,DEL03-ES,UOP02-DEL03-ES,2,1,1,2
108,654820,1474062,2025-01-12,domingo,06:20:00,BA,116,442,FEIRA DE SANTANA,Velocidade Incompatível,Saída de leito carroçável,Com Vítimas Fatais,Amanhecer,Crescente,Chuva,Dupla,Curva,Não,1177272,Caminhão-trator,M.BENZ/AXOR 2041 LS,2018,Condutor,Óbito,55,Masculino,0,0,0,1,"-12,33676101","-39,07328599",SPRF-BA,DEL02-BA,UOP01-DEL02-BA,2,1,0,2
108,654820,1474062,2025-01-12,domingo,06:20:00,BA,116,442,FEIRA DE SANTANA,Velocidade Incompatível,Saída de leito carroçável,Com Vítimas Fatais,Amanhecer,Crescente,Chuva,Dupla,Curva,Não,1177272,Caminhão-trator,M.BENZ/AXOR 2041 LS,2018,Condutor,Óbito,55,Masculino,0,0,0,1,"-12,33676101","-39,07328599",SPRF-BA,DEL02-BA,UOP01-DEL02-BA,2,1,0,2
108,654820,1474062,2025-01-12,domingo,06:20:00,BA,116,442,FEIRA DE SANTANA,Velocidade Incompatível,Saída de leito carroçável,Com Vítimas Fatais,Amanhecer,Crescente,Chuva,Dupla,Curva,Não,1177272,Caminhão-trator,M.BENZ/AXOR 2041 LS,2018,Condutor,Óbito,55,Masculino,0,0,0,1,"-12,33676101","-39,07328599",SPRF-BA,DEL02-BA,UOP01-DEL02-BA,2,1,0,2
2,652691,1468156,2025-01-01,quarta-feira,19:00:00,SC,153,"117,8",CONCORDIA,Ausência de reação do condutor,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta;Declive,Não,1173297,Caminhão-trator,DAF/XF105 FTS 460A,2020,Condutor,Lesões Leves,64,Masculino,0,1,0,0,"-27,370375","-51,993419",SPRF-SC,DEL07-SC,UOP04-DEL07-SC,2,1,1,2
4,652761,1468477,2025-01-02,quinta-feira,13:20:00,ES,262,"73,2",MARECHAL FLORIANO,Reação tardia ou ineficiente do condutor,Colisão com objeto,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Ponte;Declive;Curva,Não,1173521,Caminhão-trator,VOLVO/FH 500 6X2T,2024,Condutor,Lesões Graves,45,Masculino,0,0,1,0,"-20,41066352","-40,88380166",SPRF-ES,DEL01-ES,UOP02-DEL01-ES,2,1,1,2


In [31]:
#Verificação de dados duplicados
df.duplicated().sum()[df.duplicated().sum() > 0]

array([], dtype=int64)

In [32]:
#Verificação de valores únicos na coluna 'data_inversa'
df['data_inversa'].astype(str).str.replace(r'\d', 'D', regex=True).value_counts()

data_inversa
DDDD-DD-DD    18895
Name: count, dtype: int64

In [33]:
df['horario'].astype(str).str.replace(r'\d', 'D', regex=True).value_counts()

horario
DD:DD:DD    18895
Name: count, dtype: int64

Tratamentos

In [34]:
#Latitude e longitude como float
df['latitude'] = (df['latitude']
    .astype(str)
    .str.replace(',', '.', regex=False)
    .astype(float)
)

df['longitude'] = (
    df['longitude']
    .astype(str)
    .str.replace(',', '.', regex=False)
    .astype(float)
)
df[['latitude', 'longitude']].dtypes

latitude     float64
longitude    float64
dtype: object

In [35]:
#Preenchimento de valores nulos
df.fillna('Não informado', inplace=True)
df.isna().sum()[df.isna().sum() > 0]

Series([], dtype: int64)

In [36]:
#Substituição de valores errados pela média
#age_avg = df['idade'].mean().astype(int)

#print(f"Média original da idade: {age_avg}")

#df.loc[df['idade'] > 100, 'idade'] = age_avg

#df[df['idade'] > 100]

# ----- IGNORE ACIMA -----

df.loc[df['idade'] > 100, 'idade'] = -1

In [37]:
#ano de fabricação do veículo

df.loc[df['ano_fabricacao_veiculo'] == 0 , 'ano_fabricacao_veiculo'] = pd.NA

# -- visualização dos dados --

#df.loc[df['ano_fabricacao_veiculo'] == 'Não informado'].head(5) 
#df['ano_fabricacao_veiculo'].unique()

In [38]:
#Coluna id_veiculo sem informação
df.loc[df['id_veiculo'] == 0, 'id_veiculo'] = pd.NA

#df['id_veiculo'].info()

#df.info()

In [39]:
df['km'] = (
    df['km']
    .str.replace(',', '.', regex=False)
    .pipe(pd.to_numeric, errors='coerce')
)
df['km'].isnull().sum()

np.int64(0)

In [40]:
#Marca de veiculo não informado

df['marca'] = df['marca'].str.replace('NA/NA', 'Não Informado')

# Gold

In [41]:
#gold = df.groupby[''].size().reset_index(name='total_acidentes')
#df['tipo_pista'] == 'Dupla'

#df[(df['tipo_pista'] == 'Dupla') & (df['sentido_via'] == 'Crescente') & (df['pessoas'] >= 5)].head(5)
#gold.to_parquet('./medallion/gold/total_acidentes_uf.parquet', engine='pyarrow')

#df.to_parquet('./medallion/gold/acidentes_datatran2025.parquet', engine='pyarrow')

#caras, sei la que caralho eu tentei fazer aqui, mas ignorem

# --- IGNORE ---

df.to_csv('./medallion/gold/acidentes_datatran2025.csv', index=False)
conn = sqlite3.connect('./medallion/gold/acidentes_datatran2025.db')

df.to_sql(
    name='acidentes',
    con=conn,
    if_exists='replace',
    index=False
)

conn.close()